# Volumes and OI Data Exploration

### Import libraries and Scripts for data

In [1]:
import os
import datetime
from datetime import timedelta
import quandl
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import plotnine as p9
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore

import warnings
import functools
import wrds
# Ignore all warnings
warnings.filterwarnings("ignore")

## Data Fetching for S&P Index and Options 

In [3]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [6]:
spx_options = {} 
for year in range(2010, 2024):  # Loop from 2001 to 2023
    table_name = f"optionm.opprcd{year}"  # Generate table name dynamically
    query = f"""
    SELECT
        date, symbol, cp_flag, volume, strike_price, exdate, open_interest, impl_volatility, best_bid, best_offer, delta, gamma, vega, theta
    FROM 
        {table_name} a 
    WHERE
        a.secid = '108105' AND
        a.exdate - a.date <= 100 AND
        a.exdate - a.date >= 10 AND 
        a.volume > 0 
    """
    spx_options[year] = db.raw_sql(query, date_cols=['date'])

In [7]:
table_name = f"crsp.dsp500"  # Generate table name dynamically
query = f"""
SELECT
    *
FROM 
    {table_name} a 
WHERE
    a.caldt >= '2000-01-01' AND
    a.caldt <= '2023-12-31'
"""
sp500_underlying = db.raw_sql(query, date_cols=['date'])

In [6]:
specific_date = '2023-02-27'
table_name = f"optionm.opprcd{2023}"  # Generate table name dynamically
query = f"""
SELECT
    DISTINCT symbol
FROM 
    {table_name} a 
WHERE
    a.date = '{specific_date}'
"""
df1 = db.raw_sql(query, date_cols=['date'])

df1

,symbol
0,DGRW 230818P62000
1,IWF 230519P315000
2,SP 230721C25000
3,STAR 230317P12500
4,MAX 230519P5000
...,...
350567,XOM 230721P130000
350568,HEWG 230818C24000
350569,KOLD 240119P16000
350570,EWZ 230406C35000


In [9]:
df1['Split'] = df1['symbol'].str.split(' ').str[0]
df1['Split'].unique()

array(['DGRW', 'IWF', 'SP', ..., 'TUEM1', 'ONCS1', 'UTY'], dtype=object)

In [10]:
len(df1['Split'].unique())

6031

In [12]:
pd.DataFrame(df1['Split'].unique()).to_csv('Tickers_optionm.csv')